<a href="https://colab.research.google.com/github/M-Rezaei/Deep_Learning_Pneumonia_vs_Normal/blob/main/Deep_learning_Pneumonia_vs_normal_RGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras import backend as K
from tensorflow.keras import layers
import matplotlib.pyplot as plt




In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [3]:
main_directory = "/content/gdrive/MyDrive/Deep_Learning_projects/Pneumonia_vs_normal/data/"
train_data_dir = main_directory + "train/"
validation_data_dir = main_directory + "val/"

epochs =20
batch_size = 16
learning_rate = 0.01


train_normal = train_data_dir + "NORMAL/"
train_pneumonia = train_data_dir + "PNEUMONIA/"
img_height, img_width = 256, 256

print("\nLoading training data...")

training_data_generator = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.2,
        rotation_range=15,
        width_shift_range=0.05,
        height_shift_range=0.05)

training_iterator = training_data_generator.flow_from_directory(train_data_dir, 
                                                                class_mode='categorical', 
                                                                target_size=(img_width, img_height), 
                                                                color_mode='rgb',
                                                                batch_size=batch_size)


print("\nLoading validation data...")

validation_data_generator = ImageDataGenerator(
        rescale=1./255)

validation_iterator = validation_data_generator.flow_from_directory(validation_data_dir, class_mode='categorical', target_size=(img_width, img_height), color_mode='rgb',batch_size=batch_size)

print("\nBuilding model...")
###  model
model = Sequential()
model.add(tf.keras.Input(shape=(256, 256, 3)))
model.add(Conv2D(32, 5, strides=3, padding="same")) 
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(5, 5), strides=(5,5)))
model.add(Conv2D(1, 3, strides=1, padding="same")) 
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(tf.keras.layers.Flatten())

model.add(Dense(2,activation="softmax"))

model.summary()

print("\nCompiling model...")

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.AUC()])



print("\nTraining model...")


model.fit(
       training_iterator,
       steps_per_epoch=int(training_iterator.samples/batch_size),
       epochs=epochs,
       validation_data=validation_iterator,
       validation_steps=int(validation_iterator.samples/batch_size), verbose=1)


Loading training data...
Found 5216 images belonging to 2 classes.

Loading validation data...
Found 16 images belonging to 2 classes.

Building model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 86, 86, 32)        2432      
_________________________________________________________________
batch_normalization (BatchNo (None, 86, 86, 32)        128       
_________________________________________________________________
activation (Activation)      (None, 86, 86, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 17, 17, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 17, 17, 1)         289       
_________________________________________________________________
batch_normalization_1 (Batch (Non

In [4]:
from keras.models import load_model

model.save('chestxray_cnn_model_RGB_4.h5')

In [5]:
test_data_dir = main_directory + "test/"
test_data_generator = ImageDataGenerator(rescale=1./255)

test_iterator = test_data_generator.flow_from_directory(test_data_dir, 
                                                        class_mode='categorical', 
                                                        target_size=(img_width, img_height), 
                                                        color_mode='rgb',
                                                        batch_size=batch_size)

scores = model.evaluate(test_iterator)

print("Loss of the model: %.2f"%(scores[0]))
print("Test Accuracy: %.2f%%"%(scores[1] * 100))


Found 624 images belonging to 2 classes.
39/39 [==============================] - 147s 4s/step - loss: 0.6993 - categorical_accuracy: 0.7917 - auc: 0.8815
Loss of the model: 0.70
Test Accuracy: 79.17%


In [11]:
def predict_image(filename):
    img = load_img(filename, target_size=(img_height, img_width))
    image = keras.preprocessing.image.img_to_array(img)
    image = image / 255.0
    print(image.shape)
    #print(image.format)
    image = image.reshape( 1, 256, 256, 3)
    model = load_model('chestxray_cnn_model_RGB_4.h5')
    prediction = model.predict(image)
    #print(prediction)
    print(prediction[0][1])
    #print(prediction.shape)
    #plt.imshow(img)
    if(prediction[0][1] > 0.5):
        print("predicted: PNEUMONIA")
    else:
        print("predicted: NORMAL")
   
predict_image(test_data_dir + "PNEUMONIA/person1_virus_6.jpeg")

(256, 256, 3)
0.9999999
predicted: PNEUMONIA
